# Debugging with GDB

## Get GDB Running

Install gdb using homebrew

On Mac, we must follow the directions at https://sourceware.org/gdb/wiki/PermissionsDarwin to sign the gdb app

## Install Python extensions

http://droettboom.com/blog/2015/11/20/gdb-python-extensions/

# Install a Python with debug symbols

Probably easiest to do this with [pyenv](https://github.com/pyenv/pyenv):

In [1]:
!pyenv install --debug --keep 3.8.2

/bin/bash: pyenv: command not found


In [1]:
pwd

'/Users/rick446/src/arborian-classes/src'

In [2]:
!pyenv local 3.8.2-debug

# Running inside docker

From https://github.com/tonyOreglia/argument-counter/wiki/How-to-use-GDB-within-Docker-Container

```bash
docker run --name gdb-example -it --cap-add=SYS_PTRACE --security-opt \
    seccomp=unconfined \
    -v $(pwd):/root/app ubuntu
```

```bash
# apt update && apt install python3-dbg gdb
```

# Starting a multithreaded program

To attach to a running Python program:

```bash
$ PYTHON=$(python -c 'import sys; print(sys.executable)')
$ gdb $PYTHON $PID

(gdb) source /usr/share/gdb/auto-load/usr/bin/python3.8-gdb.py
```

in Docker, we can start the program with 

```bash
$ python3-dbg gdb-example.py &
$ gdb python3-dbg $!
```

You *may* need to enable the Python extensions:

```bash
(gdb) source /usr/share/gdb/auto-load/usr/bin/python3.8-gdb.py
```

You can also debug with core dumps (if enabled) using the following

```bash
$ ulimit -c unlimited
$ python3 gdb-example.py &
$ kill -6 $!
$ gdb python3 core
```

(You must ensure that the source files can be found if debugging a core dump)

In [1]:
%%file data/gdb-example/gdb-example.py
import time
import random
import threading

def main():
    threads = [
        threading.Thread(target=target, args=(f'T{i}',))
        for i in range(8)
    ]
    for t in threads:
        t.setDaemon(True)
        t.start()
    while True:
        time.sleep(10)
        print(f'Main thread, checking in...')
    

def target(name):
    print(f'Starting thread {name}')
    while True:
        time.sleep(30 * random.random())
        print(f'Thread {name}, checking in')
    
if __name__ == '__main__':
    main()

Overwriting data/gdb-example/gdb-example.py


Other interesting tool(s)

- https://pyrasite.readthedocs.io/en/latest/index.html